In [1]:
import pandas as pd
import matplotlib as plt

weather = pd.read_csv('dataForcast.csv', index_col="DATE" )   


In [2]:
null_pct = weather.apply(pd.isnull).sum()/weather.shape[0]


In [3]:
core_weather = weather [["PRCP", "TMAX", "TMIN"]].copy()

In [4]:
core_weather.columns = ["precipitation","max_temp", "min_temp"]

In [5]:
core_weather ["precipitation"] = core_weather ["precipitation"].fillna(0)

In [6]:
core_weather = core_weather.fillna(method="ffill")

C:\Users\Celeste\AppData\Local\Temp\ipykernel_16256\3127936549.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  core_weather = core_weather.fillna(method="ffill")


In [7]:
core_weather.apply(pd.isnull).sum()/core_weather. shape [0];

In [8]:
core_weather.apply(lambda x: (x == 9999).sum());


In [9]:
core_weather ["target"] = core_weather.shift(-1)["max_temp"]

In [10]:
core_weather = core_weather.ffill()

In [11]:
from sklearn.linear_model import Ridge
reg = Ridge(alpha=0.1)

In [12]:
predictors = ["precipitation", "max_temp","min_temp"]

In [13]:
train = core_weather.loc[:"2020-10-31"]

In [14]:
test = core_weather.loc["2021-11-01":]

In [15]:
def backtest (weather, model, predictors, start=3650, step=90):
    all_predictions = []

    for i in range (start, weather. shape[0], step):
        train = core_weather.iloc[:i, :]
        test = core_weather.iloc[i:(i+step), :]

        model.fit(train [predictors], train ["target"])
        preds = model.predict(test [predictors])
        preds=pd. Series (preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        combined ["diff"] = (combined ["prediction"] - combined ["actual"]).abs()
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [16]:
predictions = backtest (weather, reg, predictors)

In [17]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predictions["actual"], predictions["prediction"]);

In [18]:
def pct_diff(old, new) :
    return (new - old) / old
def compute_rolling (weather, horizon, col):
    label = f"rolling_{horizon}_{col}"
    weather [label] = weather [col].rolling(horizon).mean()

rolling_horizons = [3,14]

for horizon in rolling_horizons:
    for col in predictors:
        compute_rolling(core_weather, horizon, col)

In [19]:
core_weather =core_weather.iloc[14:, :]

In [20]:
core_weather = core_weather.fillna(0)

In [21]:
reg.fit(train[predictors], train["target"]);

In [22]:
predictions = reg.predict(test[predictors])

In [23]:
combined = pd.concat([test["target"], pd. Series (predictions, index=test. index)], axis=1)
combined.columns = ["actual", "predicted"]

In [24]:
reg.coef_;

In [25]:
def create_predictions (predictors, core_weather, reg):
    train = core_weather.loc[:"2020-10-31"]
    test = core_weather.loc["2020-11-01":]
    reg. fit (train [predictors], train ["target"])
    predictions = reg.predict(test [predictors])
    error = mean_absolute_error(test["target"], predictions)
    combined = pd.concat([test["target"], pd. Series (predictions, index=test.index)], axis=1)
    combined.columns = ["actual", "predictions"]
    return error, combined

create_predictions(predictors, core_weather, reg);

In [26]:
core_weather ["month_max"]= core_weather["max_temp"].rolling(30).max()

In [27]:
core_weather ["month_day_max"] = core_weather ["month_max"] / core_weather ["max_temp"]

## Predict weather for date

In [28]:
from datetime import date
from datetime import timedelta
import ipywidgets as widgets
dateInput = widgets.Text(
    placeholder='YYYY-MM-DD',
    description='Date:',
    disabled=False   
)
button = widgets.Button(
    description='Display info for date',
    disabled=False,
    button_style='info',
    tooltip='Display info for date',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

output = widgets.Textarea(
    description='Output:',
    height='20vh',
    disabled=True
)

def displayData(date_to_predict):
    date_to_predict = dateInput.value
    
    # Create a DataFrame with the specified predictors and fill missing values with zeros
    new_row = pd.DataFrame(index=[date_to_predict], columns=predictors).fillna(0)
    
    # Assuming "predictors" is a list of features
    prediction = reg.predict(new_row.values.reshape(1, -1))[0]

    # Check if the date is in the historical data
    if date_to_predict in core_weather.index:
        # Use the actual historical temperature
        max_temp = core_weather.loc[date_to_predict, "max_temp"]
        min_temp = core_weather.loc[date_to_predict, "min_temp"]
        prcp = core_weather.loc[date_to_predict, "precipitation"]
    else:
        # Estimate temperature based on the average of the same day in previous years
        same_day_temps = core_weather[pd.to_datetime(core_weather.index).dayofyear == pd.to_datetime(date_to_predict).dayofyear]
        max_temp = same_day_temps["max_temp"].mean()
        min_temp = same_day_temps["min_temp"].mean()
        prcp = same_day_temps["precipitation"].mean()

    # Check if it's going to rain
    if prcp > 0:
        precipitation_status = "It's going to rain."
    else:
        precipitation_status = "It's dry."

    output.value = "Weather forecast for {}: \nMax Temperature: {:.2f}°C\nMin Temperature: {:.2f}°C\nPrecipitation: {:.2f}\n{}".format(date_to_predict, max_temp, min_temp, prcp, precipitation_status)


button.on_click(displayData)
widgets.VBox([widgets.HBox([dateInput, button]),output])

In [29]:
from datetime import datetime

most_recent = datetime.strptime("2023-11-02","%Y-%m-%d").date()
while most_recent != (date.today() + timedelta(days=7)):
    
    # Create a DataFrame with the specified predictors and fill missing values with zeros
    new_row = pd.DataFrame(index=[most_recent], columns=predictors).fillna(0)
    
    # Assuming "predictors" is a list of features
    prediction = reg.predict(new_row.values.reshape(1, -1))[0]

     # Estimate temperature based on the average of the same day in previous years
    same_day_temps = core_weather[pd.to_datetime(core_weather.index).dayofyear == pd.to_datetime(most_recent).dayofyear]
    max_temp = same_day_temps["max_temp"].mean()
    min_temp = same_day_temps["min_temp"].mean()
    prcp = same_day_temps["precipitation"].mean()

    new_row[["precipitation","max_temp","min_temp"]] = [prcp, max_temp, min_temp]
    core_weather = pd.concat([core_weather, new_row])
    most_recent += timedelta(days=1)


C:\Users\Celeste\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\Celeste\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\Celeste\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\Celeste\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\Celeste\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  



## Weather for last and next week

In [30]:
core_weather[["precipitation","max_temp","min_temp"]].loc[(date.today() - timedelta(days=7)):]

,precipitation,max_temp,min_temp
2023-11-03,2.936508,12.119048,6.141270
2023-11-04,2.528571,11.403175,6.019048
2023-11-05,1.792063,11.322222,5.309524
2023-11-06,1.223810,10.531746,4.692063
2023-11-07,1.901587,10.928571,4.542857
2023-11-08,2.610938,11.114063,4.931250
2023-11-09,2.101587,10.920635,4.965079
2023-11-10,1.693651,10.550794,4.919048
2023-11-11,2.517460,10.528571,4.679365
2023-11-12,2.409524,10.092063,4.671429
